In [56]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import scipy.io as sc
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental import preprocessing
import time
from IPython.display import clear_output
import math 

In [57]:
c_4 = [1,-1]
c_16 = [3,1,-1,-3]
c_16r = [-3,-1,1,3]
LAMBDA = 10
EPOCHS = 5

In [58]:
import math 
cons_4 = np.dot(np.sqrt(0.5),[complex(i,j)for i in c_4 for j in c_4])
cons_16 = np.array([complex(i,j)for j in c_16 for i in c_16r])
cons_16 = cons_16/np.sqrt(np.mean(np.abs(cons_16)**2))
print(cons_4,"\n",cons_16)

[ 0.70710678+0.70710678j  0.70710678-0.70710678j -0.70710678+0.70710678j
 -0.70710678-0.70710678j] 
 [-0.9486833 +0.9486833j  -0.31622777+0.9486833j   0.31622777+0.9486833j
  0.9486833 +0.9486833j  -0.9486833 +0.31622777j -0.31622777+0.31622777j
  0.31622777+0.31622777j  0.9486833 +0.31622777j -0.9486833 -0.31622777j
 -0.31622777-0.31622777j  0.31622777-0.31622777j  0.9486833 -0.31622777j
 -0.9486833 -0.9486833j  -0.31622777-0.9486833j   0.31622777-0.9486833j
  0.9486833 -0.9486833j ]


In [106]:
def dataset(dataFile, labelFile):
    dataFile = "./communication/" + dataFile
    labelFile = "./communication/" + labelFile
    my_data = sc.loadmat(dataFile)
    my_labels = sc.loadmat(labelFile)
    my_data = my_data['Y']
    X = my_labels['L_S_x']
    myOrig = table_data(my_data, my_labels['L_Constellations'][0], X)
    mytable = assign_label(myOrig)
    return mytable


def assign_label(data):
    c_4 = [1,-1]
    c_16 = [3,1,-1,-3]
    c_16r = [-3,-1,1,3]
    cons_4 = np.dot(np.sqrt(0.5),[complex(i,j)for i in c_4 for j in c_4])
    cons_16 = np.array([complex(i,j)for j in c_16 for i in c_16r])
    cons_16 = cons_16/np.sqrt(np.mean(np.abs(cons_16)**2))
    cons4 = data[data.cons==1]
    cons4_label = np.array([[cons_4[i-1]]for i in cons4.label])
    cons16 = data[data.cons==2]
    cons16_label = np.array([[cons_16[i-1]]for i in cons16.label.to_numpy().real.astype(int)])
    data[data.cons==2].index
    data['buffer'] = 0
    data['buffer'] = 0
    data.iloc[data[data.cons==1].index, 5] = cons4_label
    data.iloc[data[data.cons==2].index, 5] = cons16_label
    data['label_real'] = data.buffer.to_numpy().real
    data['label_imag'] = data.buffer.to_numpy().imag
    myTest = data.copy()
    myTest.loc[myTest.cons == 2, 'label'] = myTest.loc[myTest.cons == 2, 'label'] + 4
    myTest.label = myTest.label - 1
    return myTest


def table_data(my_data, cons, label):
    block = my_data.shape[1]
    my_data_size = my_data.shape[0] * block
    my_data_div = my_data.T.reshape(my_data_size, )
    cons_array = np.array([[cons[i]] * my_data.shape[0] for i in range(0, block)]).reshape(my_data_size, )
    block_array = np.array([([i + 1] * my_data.shape[0]) for i in range(0, block)]).reshape(my_data_size, )
    label_array = label.T.reshape(my_data_size, )    
    test_pd = pd.DataFrame({'real': my_data_div.real, 'imag': my_data_div.imag,
                            'cons': cons_array, 'block': block_array,
                            'label': label_array})
    return test_pd

In [107]:
data1 = "hard"
data1_label = "hard_label"
data = dataset(data1, data1_label)
data

,real,imag,cons,block,label,buffer,label_real,label_imag
0,48.806980,-13.804771,2,1,15,0.948683-0.316228j,0.948683,-0.316228
1,-21.419588,-58.747407,2,1,18,0.316228-0.948683j,0.316228,-0.948683
2,61.033908,40.251547,2,1,7,0.948683+0.948683j,0.948683,0.948683
3,25.841522,72.678677,2,1,6,0.316228+0.948683j,0.316228,0.948683
4,40.576088,67.132076,2,1,7,0.948683+0.948683j,0.948683,0.948683
...,...,...,...,...,...,...,...,...
49995,28.789324,-41.060747,2,1000,18,0.316228-0.948683j,0.316228,-0.948683
49996,28.106421,-54.533358,2,1000,18,0.316228-0.948683j,0.316228,-0.948683
49997,-19.969952,-39.392252,2,1000,14,0.316228-0.316228j,0.316228,-0.316228
49998,-14.243322,5.572663,2,1000,13,-0.316228-0.316228j,-0.316228,-0.316228


In [6]:
def generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(20, use_bias=False, input_shape=[2,]))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(2))
    return model

In [7]:
generator_s = generator()

In [8]:
generator_n = generator()

In [9]:
generator_i = generator()

In [10]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(20, use_bias=False, input_shape=[2,]))
    model.add(layers.Dense(50, activation = 'sigmoid'))
    model.add(layers.Dense(1))
    return model

In [11]:
discriminator_t = make_discriminator_model()

In [12]:
discriminator_d = make_discriminator_model()

In [13]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [14]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [15]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [16]:
def identity_loss(real, fake):
    loss = tf.reduce_mean(tf.abs(real - fake))
    return LAMBDA * 0.5 * loss

In [22]:
generator_s_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_n_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_i_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_d_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_t_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [427]:
checkpoint_path = "./checkpoints/dis"

ckpt = tf.train.Checkpoint(generator_s=generator_s,
                           generator_n=generator_n,
                           generator_i=generator_i,
                           discriminator_t=discriminator_t,
                           discriminator_d=discriminator_d,
                           generator_s_optimizer=generator_s_optimizer,
                           generator_n_optimizer=generator_n_optimizer,
                           generator_i_optimizer=generator_i_optimizer,
                           discriminator_d_optimizer=discriminator_d_optimizer,
                           discriminator_t_optimizer=discriminator_t_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [27]:
@tf.function
def train_step(total, label):
    with tf.GradientTape(persistent=True) as tape:
        s = generator_s(total, training=True)
        n = generator_n(total, training=True)
        i = generator_i(total, training=True)
        gen = s + n + i
        fake_t = discriminator_t(gen, training=True)
        real_t = discriminator_t(total, training=True)
        fake_d = discriminator_d(s, training=True)
        real_d = discriminator_d(label, training=True)
        s_loss = generator_loss(s)
        n_loss = generator_loss(n)
        i_loss = generator_loss(i)
        disc_t_loss = discriminator_loss(real_t, fake_t)
        disc_d_loss = discriminator_loss(real_d, fake_d)
        identity_s_loss = identity_loss(label, s)
        identity_g_loss = identity_loss(total, gen)
        total_s_loss = 0.5 * (identity_s_loss + s_loss) + 0.5 * (identity_g_loss)
        total_n_loss = identity_g_loss + n_loss
        total_i_loss = identity_g_loss + i_loss
        print()
        
    gradients_of_s_generator = tape.gradient(total_s_loss, generator_s.trainable_variables)
    gradients_of_i_generator = tape.gradient(total_i_loss, generator_i.trainable_variables)
    gradients_of_n_generator = tape.gradient(total_n_loss, generator_n.trainable_variables)
    gradients_of_discriminator_t = tape.gradient(disc_t_loss, discriminator_t.trainable_variables)
    gradients_of_discriminator_d = tape.gradient(disc_d_loss, discriminator_d.trainable_variables)
    generator_s_optimizer.apply_gradients(zip(gradients_of_s_generator, generator_s.trainable_variables))
    generator_i_optimizer.apply_gradients(zip(gradients_of_i_generator, generator_i.trainable_variables))
    generator_n_optimizer.apply_gradients(zip(gradients_of_n_generator, generator_n.trainable_variables))
    discriminator_t_optimizer.apply_gradients(zip(gradients_of_discriminator_t, discriminator_t.trainable_variables))
    discriminator_d_optimizer.apply_gradients(zip(gradients_of_discriminator_d, discriminator_d.trainable_variables))

In [28]:
train_feature = data.loc[:,('real', 'imag')]
train_label = data.loc[:, ('label_real', 'label_imag')]

In [29]:
test_feature = tf.cast(train_feature, tf.float32)
test_label = tf.cast(train_label, tf.float32)

In [30]:
for epoch in range(EPOCHS):
    start = time.time()
    n = 0
    for i in range(len(test_feature)):
        test = tf.reshape(test_feature[i], [1,2])
        label = tf.reshape(test_label[i], [1,2])
        train_step(test, label)
        if n % 10 == 0:
            print ('.', end='')
            n+=1


.....

In [31]:
s = generator_s(test_feature, training=False)
i = generator_i(test_feature, training=False)
n = generator_n(test_feature, training=False)
gen = s + i + n
test = np.array([(test_feature - gen)])

In [61]:
a = np.array([s - test_label]).mean()
print(a)

In [54]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        layers.Dense(128, input_h
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(20)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(0.01),
                  metrics=['accuracy'])
    return model

In [74]:
generator_s(train_feature.to_numpy())

<tf.Tensor: shape=(50000, 2), dtype=float32, numpy=
array([[ 0.74356425, -0.39451778],
       [-0.63942367, -0.6913041 ],
       [ 0.69628894,  0.9595338 ],
       ...,
       [-0.59323114, -0.5585426 ],
       [-0.6560704 ,  0.2876377 ],
       [ 0.56631446,  0.91844213]], dtype=float32)>

In [84]:
size = real.size
real = np.array([s[:, 0]]).reshape(size)
imag = np.array([s[:, 1]]).reshape(size)
mlcData = pd.DataFrame({'real':real, 'imag':imag.T, 'label':symbol})

In [98]:
mlcData = pd.DataFrame({'real':real, 'imag':imag.T, 'label':symbol})

In [97]:
symbol = data['label']

In [93]:
data['test'] = 1

In [99]:
mlcData

,real,imag,label
0,0.743564,-0.394518,12
1,-0.639424,-0.691304,15
2,0.696289,0.959534,4
3,0.501473,1.278115,3
4,0.576111,1.241222,4
...,...,...,...
49995,0.607578,-0.740450,15
49996,0.569860,-0.837262,15
49997,-0.593231,-0.558543,11
49998,-0.656070,0.287638,10
